In [1]:
#import libraries
import pandas as pd
import json
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import pprint
import warnings
warnings.simplefilter('ignore')

In [2]:
url ='https://api.spoonacular.com/recipes/complexSearch?apiKey&Cuisine=Indian&addRecipeInformation=True&addRecipeNutrition=True&number=100'
res= requests.get(url).json()
#res  

{'results': [{'vegetarian': True,
   'vegan': True,
   'glutenFree': True,
   'dairyFree': True,
   'veryHealthy': True,
   'cheap': False,
   'veryPopular': True,
   'sustainable': False,
   'weightWatcherSmartPoints': 4,
   'gaps': 'no',
   'lowFodmap': False,
   'aggregateLikes': 3689,
   'spoonacularScore': 100.0,
   'healthScore': 76.0,
   'creditsText': 'Full Belly Sisters',
   'license': 'CC BY-SA 3.0',
   'sourceName': 'Full Belly Sisters',
   'pricePerServing': 112.39,
   'id': 716426,
   'title': 'Cauliflower, Brown Rice, and Vegetable Fried Rice',
   'readyInMinutes': 30,
   'servings': 8,
   'sourceUrl': 'http://fullbellysisters.blogspot.com/2012/01/cauliflower-fried-rice-more-veggies.html',
   'image': 'https://spoonacular.com/recipeImages/716426-312x231.jpg',
   'imageType': 'jpg',
   'nutrition': {'nutrients': [{'name': 'Calories',
      'amount': 191.51,
      'unit': 'kcal',
      'percentOfDailyNeeds': 9.58},
     {'name': 'Fat', 'amount': 6.36, 'unit': 'g', 'percentO

In [3]:
recipe3_df=pd.json_normalize(res,record_path=['results'])
recipe3_df.head(3)

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,weightWatcherSmartPoints,gaps,...,nutrition.flavonoids,nutrition.ingredients,nutrition.caloricBreakdown.percentProtein,nutrition.caloricBreakdown.percentFat,nutrition.caloricBreakdown.percentCarbs,nutrition.weightPerServing.amount,nutrition.weightPerServing.unit,preparationMinutes,cookingMinutes,author
0,True,True,True,True,True,False,True,False,4,no,...,"[{'name': 'Cyanidin', 'amount': 0.0, 'unit': '...","[{'id': 11090, 'name': 'broccoli', 'amount': 0...",13.63,28.46,57.91,220,g,NaN,NaN,NaN
1,True,True,False,True,True,False,True,False,19,no,...,"[{'name': 'Cyanidin', 'amount': 0.0, 'unit': '...","[{'id': 2044, 'name': 'basil', 'amount': 0.13,...",10.86,1.64,87.50,506,g,NaN,NaN,NaN
2,True,False,False,False,True,False,True,False,15,no,...,"[{'name': 'Cyanidin', 'amount': 1.21, 'unit': ...","[{'id': 9040, 'name': 'banana', 'amount': 0.25...",18.32,19.94,61.74,593,g,5.0,0.0,NaN


In [4]:
recipe3_df.drop(columns=['vegetarian', 'vegan', 'glutenFree', 'dairyFree', 'veryHealthy', 'cheap', 'veryPopular', 'sustainable', 'weightWatcherSmartPoints', 'gaps', 'lowFodmap', 'aggregateLikes', 'spoonacularScore',
                      'healthScore', 'creditsText', 'license', 'sourceName', 'pricePerServing','sourceUrl','imageType','summary','diets', 'occasions','spoonacularSourceUrl','nutrition.properties','nutrition.flavonoids','nutrition.caloricBreakdown.percentProtein','nutrition.caloricBreakdown.percentFat','nutrition.caloricBreakdown.percentCarbs','nutrition.weightPerServing.amount',
                      'nutrition.weightPerServing.unit','preparationMinutes','cookingMinutes','author','nutrition.nutrients','nutrition.ingredients'],inplace=True)
#recipe3_df.head()

In [5]:
recipe_instructions=recipe3_df[['analyzedInstructions','id']]
#recipe_instructions.head(10)

In [6]:
recipe_inst=[]
for instructions,ids in zip(list(recipe_instructions['analyzedInstructions'].values),list(recipe_instructions['id'].values)):
    #print(instructions,ids)
    #print(instructions[0]['steps'])
    instruct={}
    recipe_step=''
    try:
    #break   
        for no_steps in range(len(instructions[0]['steps'])):
            #print(no_steps)
            recipe_step+=instructions[0]['steps'][no_steps]['step']    
            #break
            #print(recipe_step)
        instruct[ids]=recipe_step
        #print(instruct)
        recipe_inst.append(instruct)
    except:
        pass

In [7]:
instruction_final=pd.DataFrame()
for dict_recipe in recipe_inst:
    df=pd.DataFrame(dict_recipe,index=dict_recipe.keys())
    df.columns=['instructions']
    #print(df)
    #break
    instruction_final=pd.concat([instruction_final,df],ignore_index=False,axis=0)

In [8]:
instruction_final.index.names = ['id']
instruction_final.head(3)

,instructions
id,
716426,Remove the cauliflower's tough stem and reserv...
715497,Take some yogurt in your favorite flavor and a...
644387,Heat the olive oil in a large pot over medium ...


# One hot encoding dishType column

In [9]:
recipe3_df['dish']=0
recipe3_df.columns

Index(['id', 'title', 'readyInMinutes', 'servings', 'image', 'cuisines',
       'dishTypes', 'analyzedInstructions', 'dish'],
      dtype='object')

In [10]:
for i in range(0,100):
    item =recipe3_df['dishTypes'][i]
    if len(item)==0:
        recipe3_df.iloc[i,8]=0
        
    else:
        #print(item[0])
        
        if any(x in ['lunch', 'dinner', 'antipasti','side dish','salad'] for x in item):
            #print(f"{i}:lunch")
            recipe3_df.iloc[i,8]=1
        if any(x in ['breakfast', 'bread'] for x in item):
            recipe3_df.iloc[i,8]=0
#recipe3_df

In [11]:
recipe3_df.drop(columns=['dishTypes'],inplace=True)
recipe3_df.head(3)

,id,title,readyInMinutes,servings,image,cuisines,analyzedInstructions,dish
0,716426,"Cauliflower, Brown Rice, and Vegetable Fried Rice",30,8,https://spoonacular.com/recipeImages/716426-31...,"[Chinese, Asian]","[{'name': '', 'steps': [{'number': 1, 'step': ...",1
1,715594,Homemade Garlic and Basil French Fries,45,2,https://spoonacular.com/recipeImages/715594-31...,[American],[],1
2,715497,Berry Banana Breakfast Smoothie,5,1,https://spoonacular.com/recipeImages/715497-31...,[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",0


In [12]:
#Getting ingredients for each recipe
#Ingredients is a list of dictionaries in nutrition key
ingredients_list=[]
recipe_idlist=[]

for recipe in res['results']:
    recipe_id =recipe['id']
    ingres=recipe['nutrition']['ingredients']
    ingri_dict={}
    for ingredient_dict in ingres:
        name=ingredient_dict['name']
        amount=ingredient_dict['amount']
        unit=ingredient_dict['unit']
        value=str(round(amount,2))+' '+unit 
        ingri_dict[name]=value
    ingredients_list.append(ingri_dict)
    recipe_idlist.append(recipe_id)    

In [13]:
ingredient_names=[]
for i in range(0,100):
    ingredients=list(ingredients_list[i])
    ingredient_names.append(ingredients)
len(ingredient_names)

100

In [14]:
recipe3_df['ingredients']=ingredients_list

In [15]:
recipe3_df['grocery']=ingredient_names

In [16]:
recipe3_df=recipe3_df.merge(instruction_final,on='id')
recipe3_df.drop(columns=['analyzedInstructions'],inplace=True)

In [17]:
recipe3_df.head(3)

,id,title,readyInMinutes,servings,image,cuisines,dish,ingredients,grocery,instructions
0,716426,"Cauliflower, Brown Rice, and Vegetable Fried Rice",30,8,https://spoonacular.com/recipeImages/716426-31...,"[Chinese, Asian]",1,"{'broccoli': '0.25 cups', 'cauliflower': '0.13...","[broccoli, cauliflower, coconut oil, cooked br...",Remove the cauliflower's tough stem and reserv...
1,715497,Berry Banana Breakfast Smoothie,5,1,https://spoonacular.com/recipeImages/715497-31...,[],0,"{'banana': '0.25 cup', 'graham cracker crumbs'...","[banana, graham cracker crumbs, soy milk, stra...",Take some yogurt in your favorite flavor and a...
2,644387,Garlicky Kale,45,2,https://spoonacular.com/recipeImages/644387-31...,[],1,"{'balsamic vinegar': '1.5 tablespoons', 'garli...","[balsamic vinegar, garlic, kale, olive oil]",Heat the olive oil in a large pot over medium ...


In [18]:
#Saving 100 recipes dataframe to a csv
recipe3_df.to_csv('./all_recipes/recipe3.csv',index=False)

In [19]:
#scraping nutrition data
nutri_list =[]
id_list=[]
for recipe in res['results']:
    recipe_id =recipe['id']
    nutrients =recipe['nutrition']['nutrients']
    nutri_dict={}
    for nutrient_dict in nutrients:
        name = nutrient_dict['name']
        amount = nutrient_dict['amount']
        unit = nutrient_dict['unit']
        #value = str(round(amount,2))+' '+unit
        key=name+' '+unit
        nutri_dict[key]=amount
    nutri_list.append(nutri_dict)
    id_list.append(recipe_id)

In [20]:
nutrition3_df= pd.DataFrame(nutri_list,index=id_list)
nutrition3_df.index.name='recipe_id'
nutrition3_df.head(3)

,Calories kcal,Fat g,Saturated Fat g,Carbohydrates g,Net Carbohydrates g,Sugar g,Cholesterol mg,Sodium mg,Protein g,Vitamin C mg,...,Vitamin B3 mg,Zinc mg,Vitamin B5 mg,Vitamin A IU,Vitamin B2 mg,Selenium µg,Vitamin E mg,Vitamin B12 µg,Vitamin D µg,Alcohol g
recipe_id,,,,,,,,,,,,,,,,,,,,,
716426,191.51,6.36,1.03,29.11,23.38,3.32,0.00,428.32,6.85,65.87,...,2.52,1.69,0.96,445.78,0.15,4.49,0.42,NaN,NaN,NaN
715594,556.62,1.02,0.21,122.29,111.17,3.50,0.00,220.80,15.18,84.46,...,8.21,1.70,1.54,166.77,0.45,22.50,NaN,NaN,NaN,NaN
715497,440.37,9.87,2.68,68.80,64.76,50.00,11.35,400.72,20.42,64.41,...,9.30,3.00,1.47,1057.73,1.02,17.21,6.24,3.75,2.83,NaN


In [21]:
print(nutrition3_df.duplicated().sum())

0


In [22]:
#saving the nutrition details of 100 recipes dataframe to csv
nutrition3_df.to_csv('./all_nutrition/nutrition3.csv',index=True)

In [23]:
#Defining dataframe for ingredients
ingres3_df=pd.DataFrame(ingredients_list,index= recipe_idlist)
ingres3_df.fillna(value=0,inplace=True)
ingres3_df.index.name='recipe_id'
ingres3_df.head(3)

,broccoli,cauliflower,coconut oil,cooked brown rice,garlic,grapeseed oil,low sodium soy sauce,peas,salt,scallion,...,beet roots,cinnamon stick,oil,red chilli flakes,whole garlic,fresh rosemary leaves,navy beans,mint leaves,chilli powder,cornflour
recipe_id,,,,,,,,,,,,,,,,,,,,,
716426,0.25 cups,0.13 head,0.13,0.38 cups,0.63 cloves,0.13,0.38 T,0.13 cup,1.0 servings,1.0 servings,...,0,0,0,0,0,0,0,0,0,0
715594,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#saving the ingredients of 100 recipes dataframe to csv
ingres3_df.to_csv('./all_ingredients/ingredients3.csv',index=True)